In [1]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
%matplotlib inline

C:\Users\kkluc\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [7]:
df=pd.read_csv(r'C:\Users\kkluc\Python Code\AB test\ab_data.csv')

df.head()

,user_id,timestamp,group,landing_page,converted
0,851104,2017-01-21 22:11:48.556739,control,old_page,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0
4,864975,2017-01-21 01:52:26.210827,control,old_page,1


In [16]:
df[['group','landing_page']].drop_duplicates()

,group,landing_page
0,control,old_page
2,treatment,new_page
22,control,new_page
308,treatment,old_page


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 294478 entries, 0 to 294477
Data columns (total 5 columns):
user_id         294478 non-null int64
timestamp       294478 non-null object
group           294478 non-null object
landing_page    294478 non-null object
converted       294478 non-null int64
dtypes: int64(2), object(3)
memory usage: 11.2+ MB


In [18]:
#找出分组错误的落地页数据，也就是本该分在新落地页的客户分在了旧的落地页

new_page=df['landing_page']=='new_page'
treatment=df['group']=='treatment'

In [21]:
(new_page!=treatment).sum()

3893

In [22]:
#为了保证数据的准确性，我们需要去除掉分组错误的数据
df_text=df[~new_page!=treatment]
df_text.shape

(290585, 5)

In [24]:
#查看数据是否又重复项
df_text.user_id.nunique()

290584

In [28]:
#查看重复值
df_text.loc[df_text.user_id.duplicated(keep=False)]

,user_id,timestamp,group,landing_page,converted
1899,773192,2017-01-09 05:37:58.781806,treatment,new_page,0
2893,773192,2017-01-14 02:55:59.590927,treatment,new_page,0


In [29]:
help(df_text.drop_duplicates)

Help on method drop_duplicates in module pandas.core.frame:

drop_duplicates(subset=None, keep='first', inplace=False) method of pandas.core.frame.DataFrame instance
    Return DataFrame with duplicate rows removed, optionally only
    considering certain columns. Indexes, including time indexes
    are ignored.
    
    Parameters
    ----------
    subset : column label or sequence of labels, optional
        Only consider certain columns for identifying duplicates, by
        default use all of the columns
    keep : {'first', 'last', False}, default 'first'
        - ``first`` : Drop duplicates except for the first occurrence.
        - ``last`` : Drop duplicates except for the last occurrence.
        - False : Drop all duplicates.
    inplace : boolean, default False
        Whether to drop duplicates in place or to return a copy
    
    Returns
    -------
    DataFrame



In [30]:
#删除重复值
df_text.drop_duplicates(subset='user_id',keep='first',inplace=True)
df_text.shape

C:\Users\kkluc\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


(290584, 5)

In [31]:
df_text.head()

,user_id,timestamp,group,landing_page,converted
0,851104,2017-01-21 22:11:48.556739,control,old_page,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0
4,864975,2017-01-21 01:52:26.210827,control,old_page,1


In [33]:
#查看落地页转化率 &用户接到新落地页概率
#-------#

#用户接到新落地页的概率
df_text.loc[df_text['landing_page']=='new_page'].shape[0]/df_text.shape[0]

0.5000619442226688

In [35]:
#用户总体转化率
print("总体转化率：")
print(df_text.converted.sum()/df_text.shape[0])

总体转化率：
0.11959708724499628


In [37]:
#用户总体转化率
print("新页面落地总体转化率：")
print(df_text.loc[df_text['landing_page']=='new_page'].converted.mean())

新页面落地总体转化率：
0.11880806551510564


In [40]:
#用户总体转化率
print("老页面落地总体转化率：")
print(df_text.loc[df_text['landing_page']!='new_page'].converted.mean())

老页面落地总体转化率：
0.1203863045004612


# 假设检验

In [62]:
import statsmodels.api as sn

C:\Users\kkluc\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Users\kkluc\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Users\kkluc\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [66]:
#旧页面购买的人数
converte_old=df_text[(df_text['group']=='control')&(df_text.converted==1)]
converte_old.shape[0]

17489

In [67]:
#访问旧页面总人数
n_old=df_text[df_text.group=='control'].shape[0]
n_old

145274

In [68]:
#新落地购买人数
converte_new=df_text.query('group=="treatment"& converted==1')
converte_new.shape[0]

17264

In [69]:
#访问新页面总人数
n_new=df_text[df_text.group!='control'].shape[0]
n_new

145310

In [88]:
z_score,p_value=sn.stats.proportions_ztest([converte_old.shape[0],converte_new.shape[0]],[n_old,n_new],
                                           
                                          alternative='larger')

In [89]:
print("z_score:%s,p_values:%f"%(z_score,p_value))

z_score:1.3109241984234394,p_values:0.094942
